In [1]:
!pip install spacy

In [1]:
# Get Harry Potter Corpus from github
import requests
res = requests.get("https://raw.githubusercontent.com/gastonstat/harry-potter-data/main/csv-data-file/harry_potter_books.csv")
from io import StringIO

import pandas as pd

TESTDATA = StringIO(res.text)

df = pd.read_csv(TESTDATA, sep=",")
df.head()

agg_df = df.groupby(['book','chapter']).text.apply(lambda x:" ".join(x))

In [3]:
import spacy
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



# Load a pretrained model for relation extraction
model_name = "Babelscape/rebel-large"  # Replace with the appropriate model name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)


/home/joung/miniconda3/envs/sqlnosql/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Load SpaCy model for NER
from spacy.cli import download
download("en_core_web_sm")
# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 6.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
from transformers import pipeline

# Load a pre-trained model fine-tuned on the MNLI dataset
nli_pipeline = pipeline("text-classification", model="roberta-large-mnli",device='cuda')

# Function to extract relations using the pretrained model
def extract_relations(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True).to(device)
    outputs = model.generate(**inputs)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

# Extract relations from each sentence

relations_list = []

for num,chap in enumerate(agg_df.index):
  print("\nRelations:")
  print(f"chap:{num}")
  #accumulated = ""
  doc = nlp(agg_df[chap])
  for sent in doc.sents:
      #accumulated += sent.text
      #if len(accumulated) < 128:
      #  continue
      relation = extract_relations(sent.text)
      triples = relation.lstrip(' ').split('  ')
      triples_cnt = len(triples) // 3
      for each in range(triples_cnt):
        temp = {}
        temp['Origin'] = chap
        temp['Subject'] = triples[each]
        temp['Object'] = triples[each+1]
        temp['Relation'] = triples[each+2]
        temp['OrigSent'] = sent.text
        inference = triples[each] + 'is' + triples[each+2] + 'of' + triples[each+1]
        premise = sent.text
        inputs = f"{premise} {nli_pipeline.tokenizer.sep_token} {inference}"

        # Get the prediction
        result = nli_pipeline(inputs)
        temp['NLIlabel'] = result[0]['label']
        temp['NLIscore'] = result[0]['score']

        relations_list.append(temp)
  with open(f'{chap}.csv', 'w') as f:
    for key in relations_list[0].keys():
        f.write(f"{key},")
    f.write("\n")
    for item in relations_list:
        for key in item.keys():
            f.write(f"{item[key]},")
        f.write("\n")
  relations_list = []
  


      #accumulated = ""


Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Relations:
chap:0


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



Relations:
chap:1

Relations:
chap:2

Relations:
chap:3

Relations:
chap:4

Relations:
chap:5

Relations:
chap:6

Relations:
chap:7

Relations:
chap:8

Relations:
chap:9

Relations:
chap:10

Relations:
chap:11

Relations:
chap:12

Relations:
chap:13

Relations:
chap:14

Relations:
chap:15

Relations:
chap:16

Relations:
chap:17

Relations:
chap:18

Relations:
chap:19

Relations:
chap:20

Relations:
chap:21

Relations:
chap:22

Relations:
chap:23

Relations:
chap:24

Relations:
chap:25

Relations:
chap:26

Relations:
chap:27

Relations:
chap:28

Relations:
chap:29

Relations:
chap:30

Relations:
chap:31

Relations:
chap:32

Relations:
chap:33

Relations:
chap:34

Relations:
chap:35

Relations:
chap:36

Relations:
chap:37

Relations:
chap:38

Relations:
chap:39

Relations:
chap:40

Relations:
chap:41

Relations:
chap:42

Relations:
chap:43

Relations:
chap:44

Relations:
chap:45

Relations:
chap:46

Relations:
chap:47

Relations:
chap:48

Relations:
chap:49

Relations:
chap:50

Relation

NER filtering

In [ ]:
import pandas as pd
import os
import re 

In [ ]:

cnt = 0
data = []
for fname in os.listdir():
    
    if fname.endswith('.csv') and fname.startswith('('):
        
        with open(fname) as f:
            
            keys = f.readline().strip().split(',')
            for line in f:
                temp = {}
                if len(line.split(',')) < 7:
                    print(fname)
                    continue
                temp['Origin'] = line.split('.csv')[0]
                temp['Subject'] = line.split(',')[2]
                temp['Object'] = line.split(',')[3]
                temp['Relation'] = line.split(',')[4]
                
               

                temp['NLIlabel'] = line.split(',')[-3]
                temp['NLIscore'] = line.split(',')[-2]

                if temp['NLIlabel'] not in ('NEUTRAL','CONTRADICTION','ENTAILMENT'):
                    continue
                #print(temp['Relation'])
                #print(temp['NLIlabel'])
                try:
                    result = re.search(f"{temp['Relation']}(.*?){temp['NLIlabel']}",line)
                except Exception as e:
                    print(e)
                    print(line)
                    continue
                if result and result.group(1):
                    temp['OrigSent'] = result.group(1).lstrip(',').rstrip(',')
                else:
                    temp['OrigSent'] = ''
                #print(temp)
                
                
                data.append(temp)
                cnt += 1

nothing to repeat at position 0
('Book 3: Prisoner of Azkaban', 'chap-22'),*ignored her,member of,*ignored her,Ron *ignored her.  ,NEUTRAL,0.8526767492294312,

nothing to repeat at position 0
('Book 2: Chamber of Secrets', 'chap-19'),I'm so glad I've got this diary to *309*,has part,*309*,I'm so glad I've got this diary to  *309*  confide in ....,ENTAILMENT,0.512889564037323,



In [ ]:
df = pd.DataFrame(data)

In [ ]:
df['NLIlabel'].value_counts()

NLIlabel
NEUTRAL          86697
ENTAILMENT       53699
CONTRADICTION    12120
Name: count, dtype: int64

In [ ]:
# only retain those rows where NLIlabel is 'ENTAILMENT' and NLIscore is greater than 0.7, in order word, valid triplets that are related to harry potter topic
df_filtered = df[(df['NLIlabel'] == 'ENTAILMENT' ) & (df['NLIscore'] > '0.7') & (df['Subject'] != df['Object']) & (df['Object'] != df['Relation']) & (df['Subject'] != df['Relation'])]

Name Entity Extraction

In [ ]:

import requests
res = requests.get("https://raw.githubusercontent.com/gastonstat/harry-potter-data/main/csv-data-file/harry_potter_books.csv")
from io import StringIO

import pandas as pd

TESTDATA = StringIO(res.text)

df_ner = pd.read_csv(TESTDATA, sep=",")
df_ner.head()

agg_df = df_ner.groupby(['book','chapter']).text.apply(lambda x:" ".join(x))

In [ ]:
corpus = " ".join(agg_df.values)

In [ ]:
# Load SpaCy model for NER
from spacy.cli import download
import spacy
download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")

# Text of length 6269985 exceeds maximum of 1000000.
batch = len(corpus) // 900000

entities_set = set()
for i in range(0, batch):
    batch_corpus = corpus[i*900000:(i+1)*900000]
    doc = nlp(batch_corpus)
    # Extract named entities
    entities_list = [(entity.text, entity.label_) for entity in doc.ents]
    filtered_entity = ['DATE','WORK_OF_ART','CARDINAL','TIME','ORDINAL']
    entities = set([s[0] for s in entities_list if s[1] not in filtered_entity ])
    for entity in entities:
        entities_set.add(entity)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 7.8 MB/s eta 0:00:00:00:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
isin = df_filtered.isin(entities_set)

In [ ]:
df_filtered[isin[['Subject','Object']].sum(axis=1) > 0].to_csv('KG.csv')

Store the triplets into neo4j aura cloud database

In [ ]:
!pip install neo4j

In [ ]:
# if you are running this code in google colab, you can use the following code to read the credentials from the user secret
# else you would need to specify the credentials manually
from google.colab import userdata
auraPassword = userdata.get('AURA2_PASSWORD')
auraUsername = userdata.get('AURA_USERNAME')
auraURI = userdata.get('AURA2_CONNECTION_URI')

In [ ]:
from neo4j import GraphDatabase
driver = GraphDatabase.driver(
    auraURI,
    auth=(auraUsername, auraPassword)
)
constraint = 'CREATE CONSTRAINT FOR (p:Object) REQUIRE p.name IS UNIQUE;'
driver.execute_query(constraint)

In [ ]:
df = pd.read_csv('KG.csv')
triples = df[['Subject','Object','Relation']]

In [ ]:

failed = []
for i in range(len(triples)):
  try:

    #print(each)
    query_string = ""
    head = 'MERGE (n:Object {name:"%s"})'%(triples.loc[i,:]['Subject'])
    query_string += head
    query_string += "\n"
    tail = 'MERGE (m:Object {name:"%s"})'%triples.loc[i,:]['Object']
    query_string += tail
    query_string += "\n"
    relation = 'MERGE (n)-[:%s]->(m)'%triples.loc[i,:]['Relation'].replace(' ','')
    query_string += relation
    #print(query_string)
    driver.execute_query(query_string)
    if (i % 100 == 0):
      print(f"{i}/{len(triples)}")
    #query_string += '\n'
    #driver.execute_query(query_string)
  except Exception as e:
    # some of the triplets might not follow the format and hence the query might fail
    print(triples.loc[i,:])
    failed.append(i)
    print(e)